In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
data = pd.read_html("https://www.latlong.net/category/cities-160-15.html")
df=data[0]
df.head()

,Place Name,Latitude,Longitude
0,"Port Harcourt, Rivers State, Nigeria",4.824167,7.033611
1,"Maiduguri, Borno, Nigeria",11.833333,13.150000
2,"Kano, Kano State, Nigeria",12.000000,8.516667
3,"Katsina, Nigeria",12.985531,7.617144
4,"Nnewi, Anambra, Nigeria",6.010519,6.910345


In [4]:

address = 'Port Harcourt, Rivers State, Nigeria'

geolocator = Nominatim(user_agent="ph_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Port Harcourt are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Port Harcourt are 4.7676576, 7.0188527.


In [5]:
# create map of all cities using latitude and longitude values
map_portharcourt = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Place Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_portharcourt)  
    
map_portharcourt

In [6]:
CLIENT_ID = 'L0AO3XJHTI4KEAWIIYHJIFM1Z13HJVQ14HKGZJD5ZZCGGSEN' # your Foursquare ID
CLIENT_SECRET = 'PEB2VCAQS54CNTHPM4HHJCLEFKMWLW1AXALML1NHLAXOWJ1V' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: L0AO3XJHTI4KEAWIIYHJIFM1Z13HJVQ14HKGZJD5ZZCGGSEN
CLIENT_SECRET:PEB2VCAQS54CNTHPM4HHJCLEFKMWLW1AXALML1NHLAXOWJ1V


In [13]:
df.loc[6, 'Place Name']

'Ikeja, Nigeria'

In [14]:
neighbourhood_latitude = df.loc[6, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = df.loc[6, 'Longitude'] # neighborhood longitude value

neighbourhood_name = df.loc[6, 'Place Name'] # neighborhood name

print('Latitude and Longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and Longitude values of Ikeja, Nigeria are 6.605874000000001, 3.3491489999999997.


In [31]:
LIMIT = 1000 # limit of number of venues returned by Foursquare API

radius = 5000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=L0AO3XJHTI4KEAWIIYHJIFM1Z13HJVQ14HKGZJD5ZZCGGSEN&client_secret=PEB2VCAQS54CNTHPM4HHJCLEFKMWLW1AXALML1NHLAXOWJ1V&v=20180605&ll=6.605874000000001,3.3491489999999997&radius=5000&limit=1000'

In [32]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f30dd5711597434c59cc3ce'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Lagos',
  'headerFullLocation': 'Lagos',
  'headerLocationGranularity': 'city',
  'totalResults': 105,
  'suggestedBounds': {'ne': {'lat': 6.650874045000046,
    'lng': 3.394365216268899},
   'sw': {'lat': 6.5608739549999555, 'lng': 3.3039327837311006}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '51fd1472498ef59d5eeba984',
       'name': 'Cold Stone Creamery',
       'location': {'address': 'Toyin Street',
        'crossStreet': 'Opebi',
        'lat': 6.595758548143397,
        'lng': 3.35265601218633,
        'labeledLatLngs': [{'label': 'display',
          'lat'

In [33]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [34]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Cold Stone Creamery,Ice Cream Shop,6.595759,3.352656
1,University of Suya,BBQ Joint,6.606512,3.349370
2,Mr. Price,Clothing Store,6.614548,3.357748
3,Best Western Plus Lagos Ikeja Hotel,Hotel,6.597282,3.354239
4,New Afrika Shrine,Performing Arts Venue,6.622918,3.356946


In [35]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [53]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Place Name', 
                  'Place Name Latitude', 
                  'Place Name Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [54]:
Nigeria_venues = getNearbyVenues(names=df['Place Name'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )



Port Harcourt, Rivers State, Nigeria
Maiduguri, Borno, Nigeria
Kano, Kano State, Nigeria
Katsina, Nigeria
Nnewi, Anambra, Nigeria
Agbor, Ika South, Nigeria
Ikeja, Nigeria
Ughelli, Nigeria
Akure, Nigeria
Calabar, Nigeria
Sapele, Delta, Nigeria
Eruwa, Oyo, Nigeria
Wudil, Nigeria
Onitsha, Nigeria
Zaria, Nigeria
Jos, Nigeria
Kaduna, Nigeria
Minna, Minna state, Nigeria
Sokoto, Nigeria
Iwo, Osun, Nigeria
Yola, Nigeria
Benin City, Nigeria
Ondo, Nigeria
Warri, Delta, Nigeria
Jos, Middle Belt, Nigeria
Enugu, Enugu, Nigeria
Ikorodu, Nigeria
Owerri, Imo, Nigeria
Bauchi, Nigeria
Bida, Nigeria
Ado Ekiti, Nigeria
Gombe, Nigeria
Sapele, Nigeria
Ofin, Nigeria
Ilorin, Nigeria
Uwheru, Nigeria
Birnin Kebbi, Nigeria
Burji, Nigeria
Surulere, Lagos, Nigeria
Abuja, Nigeria
Kaura Namoda, Nigeria
Ogbomosho, Nigeria
Eti-Osa, Nigeria
Lagos, Nigeria
Awka, Nigeria
Ibadan, Nigeria


In [55]:
print(Nigeria_venues.shape)
Nigeria_venues.head()

(61, 7)


,Place Name,Place Name Latitude,Place Name Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Port Harcourt, Rivers State, Nigeria",4.824167,7.033611,Next Cash & Carry Hypermarket,4.825448,7.035758,Supermarket
1,"Port Harcourt, Rivers State, Nigeria",4.824167,7.033611,Genesis Hotel,4.828420,7.034021,Hotel
2,"Maiduguri, Borno, Nigeria",11.833333,13.150000,Budum-Tandari Area,11.834201,13.148599,Arts & Crafts Store
3,"Maiduguri, Borno, Nigeria",11.833333,13.150000,Villager ATM Shade,11.829382,13.149668,ATM
4,"Kano, Kano State, Nigeria",12.000000,8.516667,My HALI,11.996107,8.517796,Carpet Store


In [56]:
#how many venues were returned for each city
Nigeria_venues.groupby('Place Name').count()

,Place Name Latitude,Place Name Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Place Name,,,,,,
"Abuja, Nigeria",20,20,20,20,20,20
"Agbor, Ika South, Nigeria",1,1,1,1,1,1
"Awka, Nigeria",1,1,1,1,1,1
"Bauchi, Nigeria",4,4,4,4,4,4
"Benin City, Nigeria",3,3,3,3,3,3
"Enugu, Enugu, Nigeria",1,1,1,1,1,1
"Gombe, Nigeria",1,1,1,1,1,1
"Ibadan, Nigeria",1,1,1,1,1,1
"Ikeja, Nigeria",5,5,5,5,5,5


In [57]:
#ANALYZE EACH CITY
nigeria_onehot = pd.get_dummies(Nigeria_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nigeria_onehot['Place Name'] = Nigeria_venues['Place Name'] 

# move neighborhood column to the first column
fixed_columns = [nigeria_onehot.columns[-1]] + list(nigeria_onehot.columns[:-1])
nigeria_onehot = nigeria_onehot[fixed_columns]

nigeria_onehot.head()

,Place Name,ATM,African Restaurant,Airport,Arts & Crafts Store,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Boat or Ferry,Boutique,Campground,Carpet Store,Caucasian Restaurant,Convenience Store,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Electronics Store,Fast Food Restaurant,Frozen Yogurt Shop,Golf Course,Gym,Gym / Fitness Center,Hotel,Ice Cream Shop,Lounge,Market,Mountain,Music Venue,Pizza Place,Pool,Restaurant,Smoke Shop,Spa,Steakhouse,Supermarket,Tourist Information Center
0,"Port Harcourt, Rivers State, Nigeria",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,"Port Harcourt, Rivers State, Nigeria",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Maiduguri, Borno, Nigeria",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Maiduguri, Borno, Nigeria",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Kano, Kano State, Nigeria",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [58]:
nigeria_onehot.shape

(61, 40)

In [59]:
nigeria_grouped = nigeria_onehot.groupby('Place Name').mean().reset_index()
nigeria_grouped

,Place Name,ATM,African Restaurant,Airport,Arts & Crafts Store,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Boat or Ferry,Boutique,Campground,Carpet Store,Caucasian Restaurant,Convenience Store,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Electronics Store,Fast Food Restaurant,Frozen Yogurt Shop,Golf Course,Gym,Gym / Fitness Center,Hotel,Ice Cream Shop,Lounge,Market,Mountain,Music Venue,Pizza Place,Pool,Restaurant,Smoke Shop,Spa,Steakhouse,Supermarket,Tourist Information Center
0,"Abuja, Nigeria",0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.05,0.00,0.0,0.0,0.0,0.0,0.000000,0.05,0.05,0.000000,0.00,0.00,0.05,0.0,0.05,0.05,0.00,0.15,0.0,0.05,0.00,0.0,0.0,0.0,0.05,0.3,0.05,0.05,0.05,0.0,0.00
1,"Agbor, Ika South, Nigeria",0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,1.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00
2,"Awka, Nigeria",0.0,1.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00
3,"Bauchi, Nigeria",0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.25,0.0,0.0,0.0,0.0,0.000000,0.00,0.00,0.000000,0.00,0.25,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.25,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.25
4,"Benin City, Nigeria",0.0,0.0,0.333333,0.0,0.000000,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,0.00,0.666667,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00
5,"Enugu, Enugu, Nigeria",0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.00,0.0,1.0,0.0,0.0,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00
6,"Gombe, Nigeria",0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,1.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00
7,"Ibadan, Nigeria",0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.0,0.00,0.00,0.00,1.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00
8,"Ikeja, Nigeria",0.0,0.0,0.000000,0.0,0.000000,0.2,0.0,0.000000,0.00,0.00,0.2,0.0,0.0,0.0,0.000000,0.00,0.00,0.000000,0.00,0.00,0.20,0.0,0.00,0.00,0.00,0.40,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00
9,"Ikorodu, Nigeria",0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,1.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00


In [60]:
nigeria_grouped.shape

(22, 40)

In [61]:
#each neighbourhood along with its top 5 venues
num_top_venues = 5

num_top_venues = 5

for hood in nigeria_grouped['Place Name']:
    print("----"+hood+"----")
    temp = nigeria_grouped[nigeria_grouped['Place Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abuja, Nigeria----
                  venue  freq
0            Restaurant  0.30
1                 Hotel  0.15
2  Fast Food Restaurant  0.05
3          Cupcake Shop  0.05
4                   Gym  0.05


----Agbor, Ika South, Nigeria----
                  venue  freq
0    Frozen Yogurt Shop   1.0
1              Mountain   0.0
2           Golf Course   0.0
3                   Gym   0.0
4  Gym / Fitness Center   0.0


----Awka, Nigeria----
                  venue  freq
0    African Restaurant   1.0
1  Fast Food Restaurant   0.0
2    Frozen Yogurt Shop   0.0
3           Golf Course   0.0
4                   Gym   0.0


----Bauchi, Nigeria----
                        venue  freq
0           Electronics Store  0.25
1               Boat or Ferry  0.25
2                      Market  0.25
3  Tourist Information Center  0.25
4          Athletics & Sports  0.00


----Benin City, Nigeria----
              venue  freq
0  Department Store  0.67
1           Airport  0.33
2               ATM  0.00
3

In [62]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [63]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Place Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['Place Name'] = nigeria_grouped['Place Name']

for ind in np.arange(nigeria_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nigeria_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted.head()

,Place Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Abuja, Nigeria",Restaurant,Hotel,Deli / Bodega,Lounge,Gym,Fast Food Restaurant,Bar,Golf Course,Cupcake Shop,Pool
1,"Agbor, Ika South, Nigeria",Frozen Yogurt Shop,Tourist Information Center,Dessert Shop,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
2,"Awka, Nigeria",African Restaurant,Tourist Information Center,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
3,"Bauchi, Nigeria",Tourist Information Center,Market,Boat or Ferry,Electronics Store,Smoke Shop,Bar,Convenience Store,Caucasian Restaurant,Carpet Store,Campground
4,"Benin City, Nigeria",Department Store,Airport,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique


In [67]:
#Cluster cities
kclusters = 6

nigeria_grouped_clustering = nigeria_grouped.drop('Place Name',1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nigeria_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([3, 3, 1, 3, 3, 5, 3, 2, 2, 4, 3, 0, 3, 3, 1, 0, 2, 2, 0, 3],
      dtype=int32)

In [68]:
#creating Dataframe for that includes cluster labels
city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#toronto_merged = toronto_merged.drop('Cluster Labels', axis = 1)
city_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
city_merged = city_merged.join(city_venues_sorted.set_index('Place Name'), on='Place Name')

city_merged.head() # check the last columns!

,Place Name,Latitude,Longitude,Cluster Labels,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Port Harcourt, Rivers State, Nigeria",4.824167,7.033611,2.0,0.0,Supermarket,Hotel,Department Store,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
1,"Maiduguri, Borno, Nigeria",11.833333,13.150000,3.0,0.0,ATM,Arts & Crafts Store,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
2,"Kano, Kano State, Nigeria",12.000000,8.516667,3.0,0.0,Carpet Store,Tourist Information Center,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Campground,Boutique,Bar
3,"Katsina, Nigeria",12.985531,7.617144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Nnewi, Anambra, Nigeria",6.010519,6.910345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
city_merged.head(10)

,Place Name,Latitude,Longitude,Cluster Labels,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Port Harcourt, Rivers State, Nigeria",4.824167,7.033611,2.0,0.0,Supermarket,Hotel,Department Store,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
1,"Maiduguri, Borno, Nigeria",11.833333,13.150000,3.0,0.0,ATM,Arts & Crafts Store,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
2,"Kano, Kano State, Nigeria",12.000000,8.516667,3.0,0.0,Carpet Store,Tourist Information Center,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Campground,Boutique,Bar
3,"Katsina, Nigeria",12.985531,7.617144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Nnewi, Anambra, Nigeria",6.010519,6.910345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Agbor, Ika South, Nigeria",6.264092,6.201883,3.0,0.0,Frozen Yogurt Shop,Tourist Information Center,Dessert Shop,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
6,"Ikeja, Nigeria",6.605874,3.349149,2.0,0.0,Hotel,Boutique,Fast Food Restaurant,BBQ Joint,Tourist Information Center,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground
7,"Ughelli, Nigeria",5.500187,5.993834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"Akure, Nigeria",7.250771,5.210266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"Calabar, Nigeria",4.982873,8.334503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
city_merged = city_merged.drop('Cluster Label', axis = 1)

In [71]:
city_merged.head()

,Place Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Port Harcourt, Rivers State, Nigeria",4.824167,7.033611,2.0,Supermarket,Hotel,Department Store,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
1,"Maiduguri, Borno, Nigeria",11.833333,13.150000,3.0,ATM,Arts & Crafts Store,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
2,"Kano, Kano State, Nigeria",12.000000,8.516667,3.0,Carpet Store,Tourist Information Center,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Campground,Boutique,Bar
3,"Katsina, Nigeria",12.985531,7.617144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Nnewi, Anambra, Nigeria",6.010519,6.910345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


TypeError: list indices must be integers or slices, not float

In [74]:
city_merged = city_merged.dropna()


In [75]:
city_merged.head()

,Place Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Port Harcourt, Rivers State, Nigeria",4.824167,7.033611,2.0,Supermarket,Hotel,Department Store,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
1,"Maiduguri, Borno, Nigeria",11.833333,13.150000,3.0,ATM,Arts & Crafts Store,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
2,"Kano, Kano State, Nigeria",12.000000,8.516667,3.0,Carpet Store,Tourist Information Center,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Campground,Boutique,Bar
5,"Agbor, Ika South, Nigeria",6.264092,6.201883,3.0,Frozen Yogurt Shop,Tourist Information Center,Dessert Shop,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
6,"Ikeja, Nigeria",6.605874,3.349149,2.0,Hotel,Boutique,Fast Food Restaurant,BBQ Joint,Tourist Information Center,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground


In [76]:
city_merged['Cluster Labels'] = city_merged['Cluster Labels'].astype('Int32')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [77]:
city_merged.head()

,Place Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Port Harcourt, Rivers State, Nigeria",4.824167,7.033611,2,Supermarket,Hotel,Department Store,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
1,"Maiduguri, Borno, Nigeria",11.833333,13.150000,3,ATM,Arts & Crafts Store,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
2,"Kano, Kano State, Nigeria",12.000000,8.516667,3,Carpet Store,Tourist Information Center,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Campground,Boutique,Bar
5,"Agbor, Ika South, Nigeria",6.264092,6.201883,3,Frozen Yogurt Shop,Tourist Information Center,Dessert Shop,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
6,"Ikeja, Nigeria",6.605874,3.349149,2,Hotel,Boutique,Fast Food Restaurant,BBQ Joint,Tourist Information Center,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground


In [92]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(city_merged['Latitude'], city_merged['Longitude'], city_merged['Place Name'], city_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [85]:
city_merged.loc[city_merged['Cluster Labels'] == 0, city_merged.columns[[1] + list(range(0, city_merged.shape[1]))]]

,Latitude,Place Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,5.879698,"Sapele, Delta, Nigeria",5.879698,5.700531,0,Convenience Store,Athletics & Sports,Bank,Tourist Information Center,Boutique,Deli / Bodega,Cupcake Shop,Caucasian Restaurant,Carpet Store,Campground
15,9.896527,"Jos, Nigeria",9.896527,8.858331,0,Bank,Tourist Information Center,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
22,7.100005,"Ondo, Nigeria",7.100005,4.841694,0,Bank,Tourist Information Center,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique


In [86]:
city_merged.loc[city_merged['Cluster Labels'] == 1, city_merged.columns[[1] + list(range(0, city_merged.shape[1]))]]

,Latitude,Place Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,8.142165,"Ogbomosho, Nigeria",8.142165,4.245186,1,African Restaurant,Fast Food Restaurant,Tourist Information Center,Boutique,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground
44,6.210528,"Awka, Nigeria",6.210528,7.072277,1,African Restaurant,Tourist Information Center,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique


In [87]:
city_merged.loc[city_merged['Cluster Labels'] == 2, city_merged.columns[[1] + list(range(0, city_merged.shape[1]))]]

,Latitude,Place Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4.824167,"Port Harcourt, Rivers State, Nigeria",4.824167,7.033611,2,Supermarket,Hotel,Department Store,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
6,6.605874,"Ikeja, Nigeria",6.605874,3.349149,2,Hotel,Boutique,Fast Food Restaurant,BBQ Joint,Tourist Information Center,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground
27,5.476310,"Owerri, Imo, Nigeria",5.476310,7.025853,2,Hotel,Music Venue,Tourist Information Center,Boat or Ferry,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
45,7.401962,"Ibadan, Nigeria",7.401962,3.917313,2,Hotel,Tourist Information Center,Dessert Shop,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique


In [88]:
city_merged.loc[city_merged['Cluster Labels'] == 3, city_merged.columns[[1] + list(range(0, city_merged.shape[1]))]]

,Latitude,Place Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,11.833333,"Maiduguri, Borno, Nigeria",11.833333,13.150000,3,ATM,Arts & Crafts Store,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
2,12.000000,"Kano, Kano State, Nigeria",12.000000,8.516667,3,Carpet Store,Tourist Information Center,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Campground,Boutique,Bar
5,6.264092,"Agbor, Ika South, Nigeria",6.264092,6.201883,3,Frozen Yogurt Shop,Tourist Information Center,Dessert Shop,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
12,11.794242,"Wudil, Nigeria",11.794242,8.839032,3,Caucasian Restaurant,Tourist Information Center,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Carpet Store,Campground,Boutique,Bar
21,6.339185,"Benin City, Nigeria",6.339185,5.617447,3,Department Store,Airport,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
23,5.544230,"Warri, Delta, Nigeria",5.544230,5.760269,3,Spa,Ice Cream Shop,Tourist Information Center,Boat or Ferry,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
24,9.890224,"Jos, Middle Belt, Nigeria",9.890224,8.878927,3,Pizza Place,Bakery,Tourist Information Center,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground
28,10.314159,"Bauchi, Nigeria",10.314159,9.846282,3,Tourist Information Center,Market,Boat or Ferry,Electronics Store,Smoke Shop,Bar,Convenience Store,Caucasian Restaurant,Carpet Store,Campground
31,10.283333,"Gombe, Nigeria",10.283333,11.166667,3,Mountain,Tourist Information Center,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique
38,6.500000,"Surulere, Lagos, Nigeria",6.500000,3.350000,3,Dessert Shop,Fast Food Restaurant,Gym / Fitness Center,Bank,Boutique,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store


In [89]:
city_merged.loc[city_merged['Cluster Labels'] == 4, city_merged.columns[[1] + list(range(0, city_merged.shape[1]))]]

,Latitude,Place Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,6.616865,"Ikorodu, Nigeria",6.616865,3.508072,4,Bar,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Campground,Boutique,Tourist Information Center


In [90]:
city_merged.loc[city_merged['Cluster Labels'] == 5, city_merged.columns[[1] + list(range(0, city_merged.shape[1]))]]

,Latitude,Place Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,6.459964,"Enugu, Enugu, Nigeria",6.459964,7.548949,5,Campground,Tourist Information Center,Boat or Ferry,Deli / Bodega,Cupcake Shop,Convenience Store,Caucasian Restaurant,Carpet Store,Boutique,Bar


In [91]:
city_merged.loc[city_merged['Cluster Labels'] == 6, city_merged.columns[[1] + list(range(0, city_merged.shape[1]))]]

,Latitude,Place Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [93]:
city_merged.groupby('Cluster Labels').count()

,Place Name,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,
0,3,3,3,3,3,3,3,3,3,3,3,3,3
1,2,2,2,2,2,2,2,2,2,2,2,2,2
2,4,4,4,4,4,4,4,4,4,4,4,4,4
3,11,11,11,11,11,11,11,11,11,11,11,11,11
4,1,1,1,1,1,1,1,1,1,1,1,1,1
5,1,1,1,1,1,1,1,1,1,1,1,1,1


In [94]:
cluster_city = city_merged.groupby('Cluster Labels').count()
cluster_city

,Place Name,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,
0,3,3,3,3,3,3,3,3,3,3,3,3,3
1,2,2,2,2,2,2,2,2,2,2,2,2,2
2,4,4,4,4,4,4,4,4,4,4,4,4,4
3,11,11,11,11,11,11,11,11,11,11,11,11,11
4,1,1,1,1,1,1,1,1,1,1,1,1,1
5,1,1,1,1,1,1,1,1,1,1,1,1,1


In [101]:
len(city_merged.loc[city_merged['Cluster Labels'] == 0])



3

In [102]:
len(city_merged.loc[city_merged['Cluster Labels'] == 1])



2

In [103]:
len(city_merged.loc[city_merged['Cluster Labels'] == 2])



4

In [104]:
len(city_merged.loc[city_merged['Cluster Labels'] == 3])



11

In [105]:
len(city_merged.loc[city_merged['Cluster Labels'] == 4])



1

In [106]:
len(city_merged.loc[city_merged['Cluster Labels'] == 5])



1

In [108]:
print('There are {} unique categories.'.format(len(Nigeria_venues['Venue Category'].unique())))

There are 39 unique categories.
